In [12]:
# Add project root to Python path
import os
import sys

# Get the absolute path to the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to Python path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Verify the path is added correctly
print(f"Added project root to Python path: {project_root}")

Added project root to Python path: /Users/brylie/code/remote-sensing-analysis


In [13]:
import geemap
import ee
from config.areas import get_area

Original source: https://github.com/krishnakafle/Blog_kaflekrishna/blob/main/GEE/LAI/LAI_calculations.ipynb

In [14]:
ee.Initialize()

In [ ]:
area_of_interest = get_area("finland")

In [16]:
def calculate_EVI(image: ee.Image) -> ee.Image:
    """
    Function to calculate the Enhanced Vegetation Index (EVI) from Sentinel-2 imagery.
    Args:
        image (ee.Image): Sentinel-2 image.

    Returns:
        ee.Image: Image with EVI band added.
    """
    EVI = image.expression(
        "2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))",
        {
            "NIR": image.select("B8").divide(10000),
            "RED": image.select("B4").divide(10000),
            "BLUE": image.select("B2").divide(10000),
        },
    ).rename("EVI")

    image = image.addBands(EVI)

    return image

In [17]:
def add_date(image: ee.Image) -> ee.Image:
    """Adds a date band to the image."""
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format("YYYYMMdd"))

    return image.addBands(ee.Image(img_date).rename("date").toInt())

In [18]:
START_DATE = ee.Date("2024-10-01")
END_DATE = ee.Date("2024-11-01")

# Filter Sentinel-2 data for the specified date range and area of interest
Sentinel_data = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterDate(START_DATE, END_DATE)
    .filterBounds(area_of_interest)
    .map(calculate_EVI)
)

In [19]:
def calculate_LAI(image: ee.Image) -> ee.Image:
    """Calculate Leaf Area Index (LAI) from the EVI band."""
    LAI = image.expression(
        "(3.618 * EVI - 0.118)", {"EVI": image.select("EVI")}
    ).rename("LAI")
    image = image.addBands(LAI)

    return image

In [20]:
lai_image = (
    Sentinel_data.map(calculate_LAI).map(add_date).median().clip(area_of_interest)
)

In [21]:
palette = [
    "FFFFFF",
    "CE7E45",
    "DF923D",
    "F1B555",
    "FCD163",
    "99B718",
    "74A901",
    "66A000",
    "529400",
    "3E8601",
    "207401",
    "056201",
    "004C00",
    "023B01",
    "012E01",
    "011D01",
    "011301",
]

lai_palette = {"min": 0.5, "max": 3.5, "palette": palette}
map = geemap.Map()
map.centerObject(area_of_interest, 8)
map.addLayer(lai_image.select("LAI"), lai_palette, "LAI")

map.addLayerControl()
map

Map(center=[64.23487926926077, 26.152236411247713], controls=(WidgetControl(options=['position', 'transparent_…